In [1]:
#First connect to our workspace

import azureml.core
from azureml.core import Workspace

#load the workspace from config file

ws = Workspace.from_config()
print ('Ready to use Azure{} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure1.51.0 to work with franciswrkspacefrank


In [2]:
#Train and Register our Model

#experiment going to when submitting our trained model
#Experiments is a container of trials that represents multiple model runs

from azureml.core import Experiment

#use for model registration
from azureml.core import Model

import pandas as pd
import numpy as np

# represent our job, completed tasks
import joblib
from sklearn.model_selection import train_test_split

#this is used for metric to calculated Accuracy and predicted scores.
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve



In [3]:
#create an azure ml experiments in our workspace

experiment = Experiment(workspace=ws, name="francisfirstexperiment")
run = experiment.start_logging()
print("Starting experiment",experiment.name)

Starting experiment francisfirstexperiment


In [4]:
#load the diabeted dataset
print("loading data....")
diabetes = pd.read_csv('diabetes2.csv')
diabetes.head()

loading data....


,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1142956,1,78,41,33,311,50.796392,0.420804,24,0
1,1823377,0,116,92,16,184,18.603630,0.131156,22,0
2,1916381,8,171,42,29,160,35.482247,0.082671,22,1
3,1247480,3,108,63,45,297,49.375169,0.100979,46,1
4,1516947,8,153,99,15,41,35.062139,0.116191,22,1


In [7]:
#Separate Features and labels
x, y = diabetes[['Pregnancies', 'PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values,diabetes['Diabetic'].values

In [8]:
#split data into training and test set

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
#Train a decision tree model

print('Training a decision tree model')
model = DecisionTreeClassifier().fit(x_train,y_train)

Training a decision tree model


In [11]:
#Calculate Accuracy

y_hat = model.predict(x_test)
acc = np.average(y_hat=y_test)
print('Accuracy', acc)
run.log('Accuracy', np.float128(acc))

Accuracy 0.89


In [14]:
#AUC
y_score = model.predict_proba(x_test)
auc = roc_auc_score(y_test, y_score[:,1])
print ('AUC:'+ str(auc))
run.log('AUC', np.float128(auc))

AUC:0.8750000000000001


In [16]:
#Save the trained model
model_file = 'diabetes2_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name='outputs/' + model_file,path_or_stream='./' + model_file)

In [17]:
#complete the run
run.complete()

In [18]:
#Register the model
run.register_model(model_path='outputs/diabetes2_model.pkl', model_name='diabetes2_models', tags={'Training Context':'Inline Traininng'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy':run.get_metrics()['Accuracy']})
print('model trained and Registered')


model trained and Registered


In [ ]:
#check in your model within your workspace if the model have been registered...


In [20]:
#get the informations from the Registered model

from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'Version', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t', prop_name,':',prop)
        print('\n')    


diabetes2_models Version 1
	 Training Context : Inline Traininng
	 AUC : 0.875000000000000111


	 Accuracy : 0.8900000000000000133


diabetes2_model Version 1
	 Training Context : Inline Training
	 AUC : 0.8849695199390397171


	 Accuracy : 0.89866666666666661367




In [21]:
#get the model alone by querrying the workspace model

model = ws.models['diabetes2_models']
print(model.name, 'version', model.version)

diabetes2_models version 1


In [ ]:
#lets create some scoring files and envt file we going to use for deployment with a webservices


In [43]:
import os

folder_name = 'diabetesserve'

#create a folder for the webservice files
experiement_folder = './' + folder_name
os.makedirs(experiement_folder, exist_ok=True)

print(folder_name,'folder created.')

#set the path for scoring scripts and environments files
script_file = os.path.join(experiement_folder,"scorediab.py")
env_file = os.path.join(experiement_folder,"diabeenv.yml")

diabetesserve folder created.


In [ ]:
#the web service where we deploy the model, will need some python code to load the input data, 
#get the model from workspace and generate abd return predictions, 
#we will then save this code in an an entry scripts called a scoring script 
#that will be deployed to the web servivce


In [51]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# called the when the model is loaded
def init():
    global model
    #get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes2_models')
    model = joblib.load(model_path)

#called when the request is recieved
def run(raw_data):
    #get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    #get a prediction from the model
    predictions = model.predict(data)
    #get the corresponding classnames for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes =[]
    for prediction in predictions:
        predicted_classes.append(classnames[predictions])
        #return the predictions as Json
    return json.dumps(predicted_classes)


Overwriting ./diabetesserve/scorediab.py


In [ ]:
#the web service will be hosted in a containers and the container will need to install any required python dependencies
#when it gets initialiazed. our scoring file/code require the scikitlearn and azureml-defaults packages, 
#so due to that we will create a yml that will tell the container host toinstall them into your environments

In [52]:
%%writefile $env_file
name: inference_env
dependencies:
- python=3.6.2
- scikit-learn
- pip
- pip:
  - azureml-defaults

Overwriting ./diabetesserve/diabeenv.yml


In [ ]:
#now weareready for deployments, we will deploy to a container service diabetesservicescontainer


None
